In [166]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [24]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

### Features cleaning

In [102]:
def print_percentages(array):
    for j in range(array.shape[1]):
        count_col = 0
        for i in range(array.shape[0]):
            if array[i,j] == -999:
                count_col += 1
        count_col = (count_col/array.shape[0])*100
        print(f'For the feature {j+1} : {count_col} % of missing values')

In [288]:
# standardize the matrix (not affected by undetermined values)
def standardize(tX, affected = False) :
    tX_standardized = tX.copy()
    for feature in tX_standardized.T : 
        mask = (feature != -999)
        i_mask = (feature == -999)
        if affected :
            feature[i_mask] = feature[mask].mean() # replace undetermined value by 0
            feature = (feature-feature.mean())/feature.std() # standardize
        else :
            feature[i_mask] = 0 # replace undetermined value by mean of column
            feature[mask] = (feature[mask]-feature[mask].mean())/feature[mask].std() # standardize
    tX_standardized.T[22] = tX.T[22]
    return tX_standardized

In [289]:
# standardize the matrix (affected by undetermined values)
tX_standardized = standardize(tX)
tX_standardized_af = standardize(tX, affected = True)

In [224]:
# Create mask removing columns with unvalid data
remove_na_mask = []
for j in range(tX.shape[1]):
    remove_na_mask.append(not (-999 in tX[:,j]))

In [291]:
# Create mask removing columns with unvalid data except the first one because it has only 15% on error
remove_error_mask = []
for j in range(tX.shape[1]):
    remove_error_mask.append(not (-999 in tX[:,j]))
remove_error_mask[0] = True

In [223]:
# Create mask removing all columns with unvalid or 0 data
remove_all_mask = []
for j in range(tX.shape[1]):
    remove_all_mask.append(not (-999 in tX[:,j]))
remove_all_mask[29] = False
remove_all_mask[22] = False

In [292]:
clean_cols = tX_standardized[:, remove_error_mask]

In [214]:
# for i, feature in enumerate(tX.T) :
#    print("Feature " , i+1)
#    plt.plot(feature[:1000], y[:1000], 'kp', alpha = 0.002)
#    plt.show()

In [293]:
initial_w = np.zeros(clean_cols.shape[1])
lambda_ = 50
weights, loss = least_squares(y, clean_cols)
print(weights, loss)

[ 7.08857600e-03 -2.43302526e-01 -2.17501064e-01  2.26245355e-03
  2.46368227e-01 -2.79541103e-02 -4.43537328e+02 -1.90530411e-01
  1.84650059e-01  8.60730569e+01 -9.82652830e-04 -9.88798041e-04
  8.48395551e+01  1.06768311e-03  2.85590374e-03  5.86321062e-02
  9.48365178e-04 -6.98304497e-02 -2.26372605e-01  3.75921349e+02] 0.3710326238032113


In [295]:
w_0 = np.zeros(30)
w_0[remove_error_mask] = weights
weights = w_0

## Generate predictions and save ouput in csv format for submission:

In [206]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [296]:
#standardize tX_test
tX_test_standardized = standardize(tX_test)
tX_test_standardized_af = standardize(tX_test, affected = True)

In [297]:
rmse_loss = compute_rmse(y, tX_standardized, weights)
mse_loss = compute_mse(y, tX_standardized, weights)
print('rmse ', rmse_loss)
print('mse ', mse_loss)
print('loss found ', loss)

rmse  0.8614320911171247
mse  0.37103262380321117
loss found  0.3710326238032113


In [298]:
OUTPUT_PATH = '../data/submission_file.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test_standardized)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)